In [ ]:
import gymnasium as gym

In [ ]:
env = gym.make("ALE/MsPacman-v5", render_mode="human")

In [ ]:
observation, info = env.reset()

In [ ]:
for _ in range(1000):
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    
    if terminated or truncated:
        observation, info = env.reset()

env.close()

In [ ]:
env.close()